In [1]:
from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet,COMPILER
from cimCommand.singleCmdInfo import *

In [2]:
chip=CHIP(PS("192.168.1.10", 7,debug=0))
chip.set_device_cfg(0)
chip.add_compiler("row_read_point3",filename="./assembler_code/row_read_point3.txt",encoding="utf-8")

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。


In [9]:
res = chip.read_point3(row_num_start=0,row_num_end=1,col_num_start=0,col_num_end=255,read_voltage=0.1,tg=5,gain=1,from_row=True,out_type=0)

row,col = res.shape
for i in range(row):
    for j in range(col):
        print(res[i,j])

0.0
128.0
1.0
129.0
2.0
130.0
3.0
131.0
4.0
132.0
5.0
133.0
6.0
134.0
7.0
135.0
8.0
136.0
9.0
137.0
10.0
138.0
11.0
139.0
12.0
140.0
13.0
141.0
14.0
142.0
15.0
143.0
16.0
144.0
17.0
145.0
18.0
146.0
19.0
147.0
20.0
148.0
21.0
149.0
22.0
150.0
23.0
151.0
24.0
152.0
25.0
153.0
26.0
154.0
27.0
155.0
28.0
156.0
29.0
157.0
30.0
158.0
31.0
159.0
32.0
160.0
33.0
161.0
34.0
162.0
35.0
163.0
36.0
164.0
37.0
165.0
38.0
166.0
39.0
167.0
40.0
168.0
41.0
169.0
42.0
170.0
43.0
171.0
44.0
172.0
45.0
173.0
46.0
174.0
47.0
175.0
48.0
176.0
49.0
177.0
50.0
178.0
51.0
179.0
52.0
180.0
53.0
181.0
54.0
182.0
55.0
183.0
56.0
184.0
57.0
185.0
58.0
186.0
59.0
187.0
60.0
188.0
61.0
189.0
62.0
190.0
63.0
191.0
64.0
192.0
65.0
193.0
66.0
194.0
67.0
195.0
68.0
196.0
69.0
197.0
70.0
198.0
71.0
199.0
72.0
200.0
73.0
201.0
74.0
202.0
75.0
203.0
76.0
204.0
77.0
205.0
78.0
206.0
79.0
207.0
80.0
208.0
81.0
209.0
82.0
210.0
83.0
211.0
84.0
212.0
85.0
213.0
86.0
214.0
87.0
215.0
88.0
216.0
89.0
217.0
90.0
218.0
91.0
219.

In [2]:
compiler = COMPILER()
compiler.addi("a", "zero", 0x3)
compiler.addi("b", "zero", 0x1)
compiler.add_label("c")
compiler.bge("a", "b", "c")
compiler.exit()
print(compiler)
pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
ins_str1= str(pkt)
print(ins_str1)

变量名: zero                          : 寄存器编号: 0
变量名: a                             : 寄存器编号: 1
变量名: b                             : 寄存器编号: 2

标签: c                             : 指令地址: 2

0		addi	a, zero, 3
1		addi	b, zero, 1

2	c:
2		bge	a, b, c
3		exit	

模式: 4
	帧头:        	字节码: 55 aa 04
	指令: pl_addi	字节码: 0a 03 01 00
	指令: pl_addi	字节码: 0a 01 02 00
	指令: pl_bge 	字节码: 09 00 81 02
	指令: pl_exit	字节码: 0b 00 00 00



In [2]:
for i in range(10):
    pass
print(i)

9


In [3]:
row_bank_start = 3
row_bank_end = 4
col_bank_start = 3
col_bank_end = 4

count_max = 128
pq = 0

row_bank_index_din_ram_start = 0
row_bank_index_din_ram_end = 2

col_bank_index_din_ram_start = 4
col_bank_index_din_ram_end = 6

compiler = COMPILER()
compiler.add_label("start")
compiler.addi("row_bank_start","zero",row_bank_start)
compiler.addi("row_bank_end","zero",row_bank_end)
compiler.addi("col_bank_start","zero",col_bank_start)
compiler.addi("col_bank_end","zero",col_bank_end)

compiler.addi("shift_one", "zero", 1)
compiler.addi("shift_four", "zero", 4)
compiler.addi("four", "zero", 4)
compiler.addi("count", "zero", 0)
compiler.addi("count_max", "zero", count_max)
compiler.addi("pq", "zero", pq)

# ------------------------------------------------------------
compiler.addi("row_index_start_addr","zero",row_bank_index_din_ram_start)
compiler.addi("row_index_end_addr","zero",row_bank_index_din_ram_end)

compiler.addi("row_bank_mask","zero",0x1)
compiler.sll("row_bank_mask","row_bank_mask","row_bank_start")
compiler.addi("row_bank_count","row_bank_start",0)

compiler.add_label("loop1")

# ------------------------------------------------------------
compiler.load_din_ram("row_index_count","row_index_start_addr")
compiler.load_din_ram("row_end_index","row_index_end_addr")
compiler.addi("row_index_mask", "zero", 0x1)
compiler.sll("row_index_mask", "row_index_mask", "row_index_count")
compiler.set_row_bank("row_bank_mask", "row_index_mask")

compiler.add_label("loop2")
compiler.addi("col_index_start_addr", "zero", col_bank_index_din_ram_start)
compiler.addi("col_index_end_addr", "zero", col_bank_index_din_ram_end)
compiler.addi("col_bank_mask", "zero", 0x1)
compiler.sll("col_bank_mask","col_bank_mask","col_bank_start")
compiler.addi("col_bank_count", "col_bank_start", 0)

compiler.add_label("loop3")
compiler.load_din_ram("col_index_count","col_index_start_addr")
compiler.load_din_ram("col_end_index","col_index_end_addr")
compiler.addi("col_index_mask", "zero", 0x1)
compiler.sll("col_index_mask","col_index_mask","col_index_count")

compiler.bge("col_bank_count", "four", "tia_big4")
compiler.add_label("tia_nobig4")
compiler.add("tmp_bank", "col_bank_count", "four")
compiler.jump("tia_end")
compiler.add_label("tia_big4")
compiler.sub("tmp_bank", "col_bank_count", "four")
compiler.add_label("tia_end")
compiler.sll("tmp_bank", "tmp_bank", "shift_one")

compiler.add_label("loop4")
compiler.srl("tmp_index", "col_index_count", "shift_four")
compiler.add("tia", "tmp_bank", "tmp_index")
compiler.set_col_bank("col_bank_mask", "col_index_mask")

compiler.row_read_pulse("tia", "count", "pq")
compiler.addi("count", "count", 1)
compiler.bge("count", "count_max", "return_res")
compiler.jump("jump_return")
compiler.add_label("return_res")
compiler.return_dout("count", "zero", "pq")
compiler.xori("pq", "pq", 1)
compiler.addi("count", "zero", 0)
compiler.add_label("jump_return")
compiler.bge("col_index_count", "col_end_index", "end4")
compiler.addi("col_index_count", "col_index_count", 1)
compiler.sll("col_index_mask", "col_index_mask", "shift_one")
compiler.jump("loop4")

compiler.add_label("end4")
compiler.set_col_bank("col_bank_mask", "zero")
compiler.bge("col_bank_count", "col_bank_end", "end3")
compiler.addi("col_bank_count", "col_bank_count", 1)
compiler.sll("col_bank_mask", "col_bank_mask", "shift_one")
compiler.addi("col_index_start_addr", "col_index_start_addr", 1)
compiler.addi("col_index_end_addr", "col_index_end_addr", 1)
compiler.jump("loop3")

compiler.add_label("end3")
compiler.bge("row_index_count", "row_end_index", "end2")
compiler.addi("row_index_count", "row_index_count", 1)
compiler.sll("row_index_mask", "row_index_mask", "shift_one")
compiler.jump("loop2")

compiler.add_label("end2")
compiler.set_row_bank("row_bank_mask", "zero")
compiler.bge("row_bank_count", "row_bank_end", "end1")
compiler.addi("row_bank_count", "row_bank_count", 1)
compiler.sll("row_bank_mask", "row_bank_mask", "shift_one")
compiler.addi("row_index_start_addr", "row_index_start_addr", 1)
compiler.addi("row_index_end_addr", "row_index_end_addr", 1)
compiler.jump("loop1")

compiler.add_label("end1")
compiler.return_dout("count", "zero", "pq")
compiler.exit()

# print(compiler.get_assembler_ins())
pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
ins_str1= str(pkt)
print(ins_str1)

模式: 4
	帧头:                      	字节码: 55 aa 04
	指令: pl_addi              	字节码: 0a 03 01 00
	指令: pl_addi              	字节码: 0a 04 02 00
	指令: pl_addi              	字节码: 0a 03 03 00
	指令: pl_addi              	字节码: 0a 04 04 00
	指令: pl_addi              	字节码: 0a 01 05 00
	指令: pl_addi              	字节码: 0a 04 06 00
	指令: pl_addi              	字节码: 0a 04 07 00
	指令: pl_addi              	字节码: 0a 00 08 00
	指令: pl_addi              	字节码: 0a 80 09 00
	指令: pl_addi              	字节码: 0a 00 0a 00
	指令: pl_addi              	字节码: 0a 00 0b 00
	指令: pl_addi              	字节码: 0a 02 0c 00
	指令: pl_addi              	字节码: 0a 01 0d 00
	指令: pl_sll               	字节码: 10 0d 0d 01
	指令: pl_addi              	字节码: 0a 00 0e 01
	指令: pl_load_din_ram      	字节码: 0c 00 0f 0b
	指令: pl_load_din_ram      	字节码: 0c 00 10 0c
	指令: pl_addi              	字节码: 0a 01 11 00
	指令: pl_sll               	字节码: 10 11 11 0f
	指令: pl_set_row_bank      	字节码: 12 00 0d 11
	指令: pl_addi              	字节码: 0a 04 12 00
	指令: pl_addi              	字节

In [5]:
compiler = COMPILER()
compiler.load_assembler_ins("./cimCommand/汇编.txt",encoding="utf-8")
            
print(compiler.get_assembler_ins())
pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
ins_str2= str(pkt)
print(ins_str2)

	consti	row_bank_start_c, 3
	consti	row_bank_end_c, 4
	consti	col_bank_start_c, 3
	consti	col_bank_end_c, 4
	consti	count_max_c, 128
	consti	pq_c, 0
	consti	row_index_din_ram_s_c, 0
	consti	row_index_din_ram_e_c, 2
	consti	col_index_din_ram_s_c, 4
	consti	col_index_din_ram_e_c, 6

start:	
	addi	row_bank_start, zero, row_bank_start_c
	addi	row_bank_end, zero, row_bank_end_c
	addi	col_bank_start, zero, col_bank_start_c
	addi	col_bank_end, zero, col_bank_end_c
	addi	shift_one, zero, 1
	addi	shift_four, zero, 4
	addi	four, zero, 4
	addi	count, zero, 0
	addi	count_max, zero, count_max_c
	addi	pq, zero, pq_c
	addi	row_index_start_addr, zero, row_index_din_ram_s_c
	addi	row_index_end_addr, zero, row_index_din_ram_e_c
	addi	row_bank_mask, zero, 0x01
	sll	row_bank_mask, row_bank_mask, row_bank_start
	addi	row_bank_count, row_bank_start, 0

loop1:	
	load_din_ram	row_index_count, row_index_start_addr
	load_din_ram	row_end_index, row_index_end_addr
	addi	row_index_mask, zero, 0x00000001
	sll	row_i

In [8]:
ins1 = ins_str1.split("\n")
ins2 = ins_str2.split("\n")
for i in range(len(ins1)):
    if ins1[i]!= ins2[i]:
        print(i,"   ",ins1[i],ins2[i])
print(ins_str2==ins_str1)

True


In [2]:
row_bank_start = 3
row_bank_end = 4
col_bank_start = 3
col_bank_end = 4

row_bank_index_din_ram_start = 0
row_bank_index_din_ram_end = 2

compiler = COMPILER()

compiler.addi("row_bank_start","zero",row_bank_start)
compiler.addi("row_bank_end","zero",row_bank_end)
compiler.addi("col_bank_start","zero",col_bank_start)
compiler.addi("col_bank_end","zero",col_bank_end)

# ------------------------------------------------------------
compiler.addi("row_index_start_addr","zero",row_bank_index_din_ram_start)
compiler.addi("row_index_end_addr","zero",row_bank_index_din_ram_end)

compiler.addi("row_bank_mask","zero",0x1)
compiler.sll("row_bank_mask","row_bank_mask","row_bank_start")
compiler.addi("row_bank_count","row_bank_start",0x1)

compiler.add_label("loop1")
# ------------------------------------------------------------
compiler.load_din_ram("row_index_count","row_index_start_addr")
compiler.jump("loop1")

print(compiler.get_assembler_ins())
print(compiler)
pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
print(pkt)

start:
	addi	row_bank_start, zero, 3
	addi	row_bank_end, zero, 4
	addi	col_bank_start, zero, 3
	addi	col_bank_end, zero, 4
	addi	row_index_start_addr, zero, 0
	addi	row_index_end_addr, zero, 2
	addi	row_bank_mask, zero, 1
	sll	row_bank_mask, row_bank_mask, row_bank_start
	addi	row_bank_count, row_bank_start, 1

loop1:	
	load_din_ram	row_index_count, row_index_start_addr
	jump	loop1

变量名: zero                          : 寄存器编号: 0
变量名: row_bank_start                : 寄存器编号: 1
变量名: row_bank_end                  : 寄存器编号: 2
变量名: col_bank_start                : 寄存器编号: 3
变量名: col_bank_end                  : 寄存器编号: 4
变量名: row_index_start_addr          : 寄存器编号: 5
变量名: row_index_end_addr            : 寄存器编号: 6
变量名: row_bank_mask                 : 寄存器编号: 7
变量名: row_bank_count                : 寄存器编号: 8
变量名: row_index_count               : 寄存器编号: 9

start:	0
0	addi	row_bank_start, zero, 3
1	addi	row_bank_end, zero, 4
2	addi	col_bank_start, zero, 3
3	addi	col_bank_end, zero, 4
4	addi	row_index_start_a

In [2]:
chip=CHIP(PS("192.168.1.10", 7,debug=1))
chip.set_device_cfg(0)

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: flt           	字节码: 09 00 00 0f ff
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 80
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 01 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_reset     	字节码: 0b 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_ss        	字节码: 0d 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ser_para_sel  	字节码: 0e 00 00 00 01

指令完整字节码:  55aa010900000fff
指令完整字节码:  55aa010100000080
指令完整字节码:  55aa010100000100
指令完整字节码:  55aa010b00000001
指令完整字节码:  55aa010d00000001
指令完整字节码:  55aa010e00000001
Failed to send message:
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: adc0_in       	字节码: 03 00 00 a2 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 02
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: adc1_in       	字节码: 04 00 00 a2 00
模式: 1
	帧头:               	字节码: 55 aa 01


In [3]:
crossbar = np.zeros((256,256))
crossbar[:,128] = 1
crossbar[128,:] = 1
# crossbar[128,128] = 1
res = chip.compute(crossbar=crossbar,read_voltage=0.1,tg=5,from_row=True,out_type=0)

print(res)


模式: 1
	帧头:     	字节码: 55 aa 01
	指令: gain	字节码: 10 00 00 00 01

指令完整字节码:  55aa011000000001
Failed to send message:
模式: 4
	帧头:               	字节码: 55 aa 04
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 0c
	指令: pl_dac_v      	字节码: 03 00 00 00
	指令: pl_dac_v      	字节码: 03 01 00 00
	指令: pl_dac_v      	字节码: 03 02 00 00
	指令: pl_dac_v      	字节码: 03 03 00 00
	指令: pl_dac_v      	字节码: 03 04 00 00
	指令: pl_dac_v      	字节码: 03 05 00 00
	指令: pl_dac_v      	字节码: 03 06 00 00
	指令: pl_dac_v      	字节码: 03 07 00 00
	指令: pl_dac_v      	字节码: 03 08 00 00
	指令: pl_dac_v      	字节码: 03 09 00 00
	指令: pl_dac_v      	字节码: 03 0a 00 00
	指令: pl_dac_v      	字节码: 03 0b 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ins_num       	字节码: 1c 00 00 00 0c
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 80 00

指令完整字节码:  55aa040000000c03000000030100000302000003030000030400000305000003060000030700000308000003090000030a0000030b0000
指令完整字节码:  55aa011c0000000c
指令完整字节码:  55aa010100008000
Failed

AttributeError: 'NoneType' object has no attribute 'hex'

In [3]:
crossbar = np.zeros((256,256))
tg=np.ones((256,256))
tg[5,128]=2
tg[7,128]=1.5
crossbar[:,128] = 1
# crossbar[128,:] = 1
# crossbar[128,128] = 1
res = chip.write_point2(crossbar=crossbar,write_voltage=0.1,tg=tg,pulse_width=1e-3,set_device=True)

# print(res[:,128])

# print(res[128,:])

模式: 4
	帧头:               	字节码: 55 aa 04
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 0c
	指令: pl_dac_v      	字节码: 03 00 00 00
	指令: pl_dac_v      	字节码: 03 01 00 00
	指令: pl_dac_v      	字节码: 03 02 00 00
	指令: pl_dac_v      	字节码: 03 03 00 00
	指令: pl_dac_v      	字节码: 03 04 00 00
	指令: pl_dac_v      	字节码: 03 05 00 00
	指令: pl_dac_v      	字节码: 03 06 00 00
	指令: pl_dac_v      	字节码: 03 07 00 00
	指令: pl_dac_v      	字节码: 03 08 00 00
	指令: pl_dac_v      	字节码: 03 09 00 00
	指令: pl_dac_v      	字节码: 03 0a 00 00
	指令: pl_dac_v      	字节码: 03 0b 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ins_num       	字节码: 1c 00 00 00 0c
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 80 00

指令完整字节码:  55aa040000000c03000000030100000302000003030000030400000305000003060000030700000308000003090000030a0000030b0000
指令完整字节码:  55aa011c0000000c
指令完整字节码:  55aa010100008000
Failed to send message:
模式: 1
	帧头:          	字节码: 55 aa 01
	指令: pulse_cyc	字节码: 1b 00 01 86 a0

指令完整字节码:  55aa011b00018

In [21]:
crossbar = np.zeros((256,256))
crossbar[:,128] = 1
crossbar[128,:] = 1
# crossbar[128,128] = 1
res = chip.read_point2(crossbar=crossbar,read_voltage=0.1,tg=5,from_row=False,out_type=0)

print(res[:,128])

print(res[128,:])

模式: 4
	帧头:               	字节码: 55 aa 04
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 0c
	指令: pl_dac_v      	字节码: 03 00 00 00
	指令: pl_dac_v      	字节码: 03 01 00 00
	指令: pl_dac_v      	字节码: 03 02 00 00
	指令: pl_dac_v      	字节码: 03 03 00 00
	指令: pl_dac_v      	字节码: 03 04 00 00
	指令: pl_dac_v      	字节码: 03 05 00 00
	指令: pl_dac_v      	字节码: 03 06 00 00
	指令: pl_dac_v      	字节码: 03 07 00 00
	指令: pl_dac_v      	字节码: 03 08 00 00
	指令: pl_dac_v      	字节码: 03 09 00 00
	指令: pl_dac_v      	字节码: 03 0a 00 00
	指令: pl_dac_v      	字节码: 03 0b 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ins_num       	字节码: 1c 00 00 00 0c
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 80 00

指令完整字节码:  55aa040000000c03000000030100000302000003030000030400000305000003060000030700000308000003090000030a0000030b0000
指令完整字节码:  55aa011c0000000c
指令完整字节码:  55aa010100008000
Failed to send message:
模式: 5
	帧头:               	字节码: 55 aa 05
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码